In [1]:
# the framework we rely on, see: https://dspy.ai/ and https://dspy.ai/tutorials/agents/ 
import dspy

# I created 2 github tools, that the agent can use
from summarizer.github_tools import read_github_file, search_github

# We reuse the tools created in part 1, as these can be used by the agent to use when they think its needed
from summarizer.main import generate_summary
from summarizer.main import main as get_diff

from summarizer.argparsing import DEFAULT_PROJECT_FOLDER

/Users/dnoordman/ai-workshop/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
/Users/dnoordman/ai-workshop/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# MLFlow is a tool that can be used to track ML experiments and evaluate model training.
# Currently its upgraded with `traces` that is super cool to use and track AI agents.

import mlflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("ai-workshop2")
mlflow.dspy.autolog()

In [3]:
# We create a wrapper around the generate_summary function, so that it can be used by the agent.
# The main/only improvement is that we add typehinting, so that the agent knows what goes in/out.
def generate_summary_tool(diff: str) -> str:
    return generate_summary(diff)


In [4]:
# create dspy LM object
lm = dspy.LM("ollama_chat/llama3.2", api_base="http://localhost:11434", api_key="")

# configure this as the default LLM to use in dspy functions
dspy.configure(lm=lm)

# create instructions for the agent
instructions = "Find all code references that might be impacted by the code change in the code_diff."

# Dspy relies on the concept of `signature` to define the input and output of the agent.
signature = dspy.Signature("code_diff -> code_references: list[str]", instructions)

# create the agent, and define which tools it can use
react = dspy.ReAct(signature, tools=[generate_summary_tool, search_github, read_github_file], max_iters=2)

INFO:root:Using args: Namespace(base_branch='main', feature_branch='newbranch', project_folder='/Users/dnoordman/connectedbrewery/terraform-aws-uc-d06a/.', log_level='INFO')


In [5]:
# Now lets start!
# We will use our previous project to get the PR diff and pass this in the agent to complete the `instructions`

diff = get_diff(
    args_list=["--project_folder", DEFAULT_PROJECT_FOLDER, "--log-level", "INFO"],
    summarize=False,
)
r = react(code_diff=diff)

INFO:root:Using args: Namespace(base_branch='main', feature_branch='newbranch', project_folder='/Users/dnoordman/connectedbrewery/terraform-aws-uc-d06a/.', log_level='INFO')
